## Environment Setup

In [1]:
from argparse import Namespace
from lifelines.utils import concordance_index
import numpy as np
import os
from pysurvival.models.simulations import SimulationModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from utils import *
from models import *

### models.py
Code from Hassan

In [2]:
# class BasicModel(nn.Module):
#     '''Module class builds network according to config'''
#     def __init__(self, activation='SELU'):
#         ''' Initialize BasicModel class
        
#         Args:
#             activation: string, name of activation function to use
        
#         Returns:
#             torch.nn Module object, built sequential network
#         '''
#         super(BasicModel, self).__init__()
        
#         # Set some defaults for network arguments
#         # Fraction of input units to drop in dropout layer
#         self.drop = 0.375
#         # Flag to in/exclude normalization layer
#         self.norm = True
#         # Default dimensions of fully connected layers
#         self.dims = [10, 4, 1]
#         # Activation type to use
#         self.activation = activation
        
#         # Build network using class function (below)
#         self.model = self._build_network()
    
#     def _build_network(self):
#         '''Build network according to parameters'''
#         layers =  []
        
#         for i in range(len(self.dims)-1):
#             if i and self.drop is not None:
#                 # Add dropout layer
#                 layers.append(nn.Dropout(self.drop))
            
#             # Add fully connected layer
#             layers.append(nn.Linear(self.dims[i], self.dims[i+1]))
            
#             if self.norm:
#                 # Add batch normalize layer
#                 layers.append(nn.BatchNorm1d(self.dims[i+1]))
                
#             # Add activation layer
#             # eval creates the proper format of the activation to get from nn
#             layers.append(eval('nn.{}()'.format(self.activation)))
            
#         # Build sequential network from list of layers created in for loop
#         return nn.Sequential(*layers)
    
#     def forward(self, X):
#         ''' Forward propagation through network
        
#         Args:
#             X: data to pass through network
        
#         Returns:
#             Output of model (risk prediction)
#         '''
#         return self.model(X)
    

# class NegativeLogLikelihood(nn.Module):
#     '''Negative log likelihood loss function from Katzman et al. (2018) DeepSurv model (equation 4)'''
    
#     def __init__(self, gpu='None'):
#         ''' Initialize NegativeLogLikelihood class
        
#         Args:
#             gpu: string, what kind of tensor to use for loss calculation 
#         '''
#         super(NegativeLogLikelihood, self).__init__()
# #         self.L2_reg = 0
#         self.reg = Regularization(order=2, weight_decay=0)
#         self.device = gpu
        
#     def forward(self, risk_pred, y, e, model):
#         # Think this is getting set of patients still at risk of failure at time t???
#         mask = torch.ones(y.shape[0], y.shape[0], device=self.device)
#         mask[(y.T - y) > 0] = 0
        
#         log_loss = torch.exp(risk_pred) * mask
#         log_loss = torch.sum(log_loss, dim=0) / torch.sum(mask, dim=0)
#         log_loss = torch.log(log_loss).reshape(-1,1)
        
#         neg_log_loss = -torch.sum((risk_pred - log_loss) * e) / torch.sum(e)
        
#         l2_loss = self.reg(model)
        
#         return neg_log_loss + l2_loss
    

# class Regularization(object):
#     def __init__(self, order, weight_decay):
#         ''' Initialize Regularization class
        
#         Args:
#             order: int, norm order number
#             weight_decay: float, weight decay rate
#         '''
#         super(Regularization, self).__init__()
#         self.order = order
#         self.weight_decay = weight_decay
        
#     def __call__(self, model):
#         ''' Calculates regularization(self.order) loss for model
        
#         Args:
#             model: torch.nn Module object
        
#         Returns:
#             reg_loss: torch.Tensor, regularization loss
#         '''
#         reg_loss = 0
#         for name, w in model.named_parameters():
#             if 'weight' in name:
#                 reg_loss = reg_loss + torch.norm(w, p=self.order)
            
#             reg_loss = self.weight_decay * reg_loss
#             return reg_loss

        

## utils.py
Code from Hassan

In [3]:
# class SurvivalDataset(Dataset):
#     def __init__(self, dataset, args):
#         '''Initialize SurvivalDataset class
        
#         Args:
#             dataset: pandas.Dataframe, Contains covariates, time of event (T), and event indicator (E) values.
#             T and E must be the final two columns
#             args: Namespace, 
#         '''
#         # Get covariates out of dataframe (args.covariates is num of columns containing covariates)
#         self.X = dataset.iloc[:, 0:args.covariates].values
#         # Get time and event indicator columns out of dataframe
#         self.data = list(zip(dataset.time, dataset.event))
#         self.len = len(dataset)
#         print('=> load {} samples'.format(self.len))
#         # Normalize covariate data with class function
#         self._normalize()
        
#     def _normalize(self):
#         '''Normalize X data (covariates) (transform values to range between 0 and 1)'''
#         self.X = (self.X - self.X.min(axis=0)) / (self.X.max(axis=0) - self.X.min(axis=0))

#     def __getitem__(self, item):
#         '''Getter for single data piece

#         Args:
#             item: int, index of data to retrieve

#         Returns:
#             X_tensor: torch.Tensor, covariate values for data item
#             y_tensor: torch.Tensor, time of event value for data item
#             e_tensor: int torch.Tensor, event indicator value for data item
#         '''
#         y, e = self.data[item]
#         X_tensor = torch.from_numpy(self.X[item])
#         y_tensor = torch.Tensor([y])
#         e_tensor = torch.Tensor([e]).int

#     def __len__(self):
#         return self.len
        

# def save_error(train_ci, val_ci, coxLoss, stratLoss, variance, epoch, slname):
#     '''Save training and validation statistics to csv file
    
#     Args:
#         train_ci: float, training concordance index for this epoch
#         val_ci: float, validation concordance index for this epoch
#         coxLoss: 
#         stratLoss:
#         variance:
#         epoch: int, epoch these stats are from
#         slname: string, filename 
#     '''
#     if epoch == 0:
#         # Create file for first epoch
#         f = open(slname, 'w')
#         f.write('epoch,coxLoss,stratLoss,trainCI,valCI,variance\n')
#         f.write('{},{:.4f},{:.4f},{:.4f},{:.4f},{}\n'.format(epoch, coxLoss, stratLoss, train_ci, val_ci, variance))
#         f.close()
#     else:
#         f = open(slname, 'a')
#         f.write('{},{:.4f},{:.4f},{:.4f},{:.4f},{}\n'.format(epoch, coxLoss, stratLoss, train_ci, val_ci, variance))
#         f.close()


# def c_index(risk_pred, y, e):
#     '''Calculate c-index
    
#     Args:
#         risk_pred: np.ndarray or torch.Tensor, model prediction
#         y: np.ndarray or torch.Tensor, times of event e
#         e: np.ndarray or torch.Tensor, event indicator
    
#     Returns:
#         c_index: float, concordance index 
#     '''
#     # Convert risk_pred, y, and e from torch.Tensor to np.ndarray if not already
#     if not isinstance(risk_pred, np.ndarray):
#         risk_pred = risk_pred.detach().cpu().numpy()
        
#     if not isinstance(y, np.ndarray):
#         y = y.detach().cpu().numpy()
        
#     if not isinstance(e, np.ndarray):
#         e = e.detach().cpu().numpy()
    
#     return concordance_index(y, risk_pred, e)


# class AverageMeter(object):
#     '''Computes and stores average and current value'''
#     def __init__(self):
#         self.reset()
    
#     def reset(self):
#         self.val = 0
#         self.avg = 0
#         self.sum = 0
#         self.count = 0
        
#     def update(self, val, n=1):
#         self.val = val
#         self.sum += val * n
#         self.count += n
#         self.avg = self.sum / self.count


# def adjust_learning_rate(optimizer, epoch, lr, lr_decay):
#     '''Adjust learning rate according to (epoch, lr, and lr_decay_rate)
    
#     Args:
#         optimizer: torch.optim object, 
#         epoch: int, epoch number
#         lr: float, initial learning rate 
#         lr_decay_rate: float, decay rate to apply to learning rate
    
#     Returns:
#         lr: float, updated learning rate
#     '''
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr / (1+epoch*lr_decay_rate)
    
#     return optimizer.param_groups[0]['lr']


### train.py
Code from Hassan

In [2]:
# Arguments for network
args = Namespace(activation = 'SELU',
                 batch_size = 4000,
                 covariates = 10, 
                 decay_interval = 400,
                 development = 0,
                 dropout = 0.3,
                 epochs = 500,
                 lib = '',
                 lr = 0.001,
                 out = 1,
                 strat = 'none',# not sure if I actually need this one
                 weight_decay = 0.0001
                )

best_acc = 0
# Where to allocate all the Tensors (can be 'cpu' or 'coda')
gpu = torch.device("cpu")

# Setting up output path from model training
root_output = '/Users/katyscott/Documents/ICC/Code/cox_experiments'

if args.development == 1:
    save_path = 'test'
else:
    save_path = '{}_{}lr_{}b_'.format(args.activation,args.lr,args.batch_size)
    
out_dir = os.path.join(root_output, save_path)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Build network 
model = BasicModel(args.activation).to(gpu)

# Loss function
criterion = NegativeLogLikelihood(gpu)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

Simulated Data creation

In [3]:
# generate random survival times with exp. distribution
sim = SimulationModel(survival_distribution='exponential',
                      risk_type = 'Linear',
                      censored_parameter = 6,
                      alpha = 1,
                      beta = 5)

train_samples = sim.generate_data(num_samples = 4000,
                                  num_features = args.covariates,
                                  feature_weights = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

val_samples = sim.generate_data(num_samples = 500,
                                num_features = args.covariates,
                                feature_weights = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

train_dataset = SurvivalDataset(train_samples, args)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_dataset.__len__())

val_dataset = SurvivalDataset(val_samples, args)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_dataset.__len__())


Number of data-points: 4000 - Number of events: 3138.0
Number of data-points: 500 - Number of events: 381.0
=> load 4000 samples
=> load 500 samples


In [4]:
for X, y, e in train_loader:
    print(y)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [24]:
for epoch in range(0, args.epochs):
    coxLossMeter = AverageMeter()
    stratLossMeter = AverageMeter()
    ciMeter = AverageMeter()
    varMeter = AverageMeter()
    
    # Training
    model.train()
    for X, y, e in train_loader:
        # Get risk prediction from network
        risk_pred = model(X.float().to(gpu))
        
        # Calculate neg. log likelihood
        cox_loss = criterion(-risk_pred, y.to(gpu), e.to(gpu), low, high)
        strat_loss = torch.Tensor([0])
        train_loss = cox_loss
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        coxLossMeter.update(cox_loss.item(), y.size(0))
        stratLossMeter.update(strat_loss.item(), y.size(0))
        varMeter.update(risk_pred.var(), y.size(0))
        
        # Calculate c index
        train_c = c_index(risk_pred, y, e)
        ciMeter.update(train_c.item(), y.size(0))
    
    # Validation
    model.eval()
    ciValMeter = AverageMeter()
    for X, y, e in val_loader:
        risk_pred = model(X.float().to(gpu))
        val_c = c_index(risk_pred, y, e)
        ciValMeter.update(val_c.item(), y.size(0))
    
    print('Epoch: {} \t Train Loss: {:.4f} \t Train CI: {:.3f} \t Val CI: {:.3f}'.format(epoch, train_loss, train_c, val_c))
    save_error(ciMeter.avg, ciValMeter.avg, coxLossMeter.avg, stratLossMeter.avg, varMeter.avg, epoch, os.path.join(out_dir, 'convergence.csv'))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [ ]:
#